In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [2]:
full_train_orig = pd.read_csv("../Data/application_train.csv")
test =  pd.read_csv("../Data/application_test.csv")
bureau = pd.read_csv("../Data/bureau.csv")

In [3]:
'''
Removing columns with more than 100 null values (filters out 62 columns out of 122 columns)
And not useful (based on EDA) which removes reduces feature size to 43)
'''

null_columns = full_train_orig.columns[full_train_orig.isnull().sum().values > 100].values.tolist()
useless_columns = ["FLAG_DOCUMENT_2", "FLAG_DOCUMENT_4", "FLAG_DOCUMENT_5", "FLAG_DOCUMENT_7"
                  ,'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

In [4]:
### Manually removing columns which doesn't make sense (based on EDA)
full_train = full_train_orig.drop(null_columns+useless_columns, axis = 1)

In [5]:
train_bureau = pd.merge(full_train_orig, bureau, how="left", on="SK_ID_CURR")

In [6]:
bureau.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [7]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [10]:
#cat_columns_all = set(full_train.columns) - set(full_train._get_numeric_data().columns.tolist())

In [11]:
# cat_feats = ["CODE_GENDER", "NAME_CONTRACT_TYPE", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", 
#              "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "OCCUPATION_TYPE",
#              "NAME_HOUSING_TYPE", "FLAG_DOCUMENT_3"]
# num_feats = ["AMT_INCOME_TOTAL", "CNT_CHILDREN", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "DAYS_BIRTH",
#              "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_LAST_PHONE_CHANGE", "AMT_REQ_CREDIT_BUREAU_YEAR", 
#             "REGION_POPULATION_RELATIVE", ]

In [12]:
full_train_y = full_train.TARGET.values
full_train = full_train.drop(["TARGET"], axis = 1)
full_train = full_train.set_index("SK_ID_CURR")
num_feats = full_train._get_numeric_data().columns.values.tolist()
cat_feats = list(set(full_train.columns.values) - set(num_feats))

In [13]:
## Categorical Features - Train
train_cat= full_train[cat_feats]
train_cat = pd.get_dummies(train_cat)

## Numerical Features - Train
train_num = full_train[num_feats]
# train[num_feats].isnull().any() # Checking if the column has any null value
#train_num = train_num.fillna((train_num.mean()), inplace=True)

## Categorical Features - Test
test_cat = test[cat_feats]
test_cat = pd.get_dummies(test_cat)

## Numerical Features - Test
test_num = test[num_feats]
test_num = test_num.fillna(test_num.mean())

# ## Standardize numerical features
# std = StandardScaler().fit(train_num)
# train_num = std.transform(train_num)
# test_num = std.transform(test_num)


In [14]:
full_train_feats = pd.concat([train_num, train_cat], axis=1)
test_feats = pd.concat([test_num, test_cat], axis=1)

In [15]:
full_train_feats = full_train_feats.apply(lambda x: x.fillna(x.mean()),axis=0)
test_feats = test_feats.apply(lambda x: x.fillna(x.mean()),axis=0)

In [16]:
train_X, valid_X, train_y, valid_y = train_test_split(full_train_feats, full_train_y, train_size = 0.8)

In [22]:
sm = SMOTE(random_state=12, ratio = 1.0)
train_X_res, train_y_res = sm.fit_sample(train_X, train_y)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [27]:
train_X.shape

(31065, 128)

In [29]:
train_X_res = pd.DataFrame(train_X_res, columns=train_X.columns)

In [30]:
### RF classifier
params_rf={
    'max_depth': [20, 40, 60], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'n_estimators': [100, 300, 500], #[1000,2000,3000]
}

rf_clf = RandomForestClassifier()
rf = GridSearchCV(rf_clf,
                  params_rf,
                  cv=3,
                  scoring="roc_auc",
                  n_jobs=1,
                  verbose=2)
rf.fit(train_X_res.drop("CODE_GENDER_XNA", axis=1), train_y_res)
best_est_rf = rf.best_estimator_
print(best_est_rf)
valid_probs_rf = rf.predict_proba(valid_X)[:,1]
valid_preds_rf = rf.predict(valid_X)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=20, n_estimators=100 ..................................
[CV] ................... max_depth=20, n_estimators=100, total=   7.6s
[CV] max_depth=20, n_estimators=100 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s


[CV] ................... max_depth=20, n_estimators=100, total=   8.6s
[CV] max_depth=20, n_estimators=100 ..................................
[CV] ................... max_depth=20, n_estimators=100, total=   9.2s
[CV] max_depth=20, n_estimators=300 ..................................
[CV] ................... max_depth=20, n_estimators=300, total=  25.0s
[CV] max_depth=20, n_estimators=300 ..................................
[CV] ................... max_depth=20, n_estimators=300, total=  23.5s
[CV] max_depth=20, n_estimators=300 ..................................
[CV] ................... max_depth=20, n_estimators=300, total=  23.4s
[CV] max_depth=20, n_estimators=500 ..................................
[CV] ................... max_depth=20, n_estimators=500, total=  36.3s
[CV] max_depth=20, n_estimators=500 ..................................
[CV] ................... max_depth=20, n_estimators=500, total=  38.5s
[CV] max_depth=20, n_estimators=500 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 12.5min finished


NameError: name 'best_est' is not defined

In [42]:
# trainrf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=40, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)

# trainrf.fit(train_X.drop("CODE_GENDER_XNA", axis=1), train_y)
# valid_probs_rf = trainrf.predict_proba(valid_X.drop("CODE_GENDER_XNA", axis=1))[:,1]
# valid_preds_rf = trainrf.predict(valid_X.drop("CODE_GENDER_XNA", axis=1))

In [43]:
print(accuracy_score(valid_y, valid_preds_rf))
print(roc_auc_score(valid_y, valid_probs_rf))

0.917213853483
0.662401882494


In [44]:
params={
    'max_depth': [3, 5], #[3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.6, 0.8], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5, 0.7], #[0.5,0.6,0.7,0.8],
    'n_estimators': [300, 500], #[1000,2000,3000]
    'reg_alpha': [0.01, 0.05] #[0.01, 0.02, 0.03, 0.04]
}

xgb_clf = xgb.XGBClassifier(missing=9999999999)
rs = GridSearchCV(xgb_clf,
                  params,
                  cv=3,
                  scoring="roc_auc",
                  n_jobs=1,
                  verbose=2)
rs.fit(train_X_res.drop("CODE_GENDER_XNA", axis=1), train_y_res)
best_est = rs.best_estimator_
print(best_est)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] colsample_bytree=0.5, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.5, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300, total=  20.8s
[CV] colsample_bytree=0.5, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.0s remaining:    0.0s


[CV]  colsample_bytree=0.5, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300, total=  23.1s
[CV] colsample_bytree=0.5, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.5, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300, total=  22.3s
[CV] colsample_bytree=0.5, reg_alpha=0.01, subsample=0.8, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.5, reg_alpha=0.01, subsample=0.8, max_depth=3, n_estimators=300, total=  27.0s
[CV] colsample_bytree=0.5, reg_alpha=0.01, subsample=0.8, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.5, reg_alpha=0.01, subsample=0.8, max_depth=3, n_estimators=300, total=  28.3s
[CV] colsample_bytree=0.5, reg_alpha=0.01, subsample=0.8, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.5, reg_alpha=0.01, subsample=0.8, max_depth=3, n_estimators=300, total=  22.1s
[CV] colsample_bytree=0.5, reg_alpha=0.05, subsample=0.6, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.5, reg_alp

[CV]  colsample_bytree=0.5, reg_alpha=0.05, subsample=0.6, max_depth=5, n_estimators=500, total=  55.6s
[CV] colsample_bytree=0.5, reg_alpha=0.05, subsample=0.8, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.5, reg_alpha=0.05, subsample=0.8, max_depth=5, n_estimators=500, total=  44.4s
[CV] colsample_bytree=0.5, reg_alpha=0.05, subsample=0.8, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.5, reg_alpha=0.05, subsample=0.8, max_depth=5, n_estimators=500, total=  44.4s
[CV] colsample_bytree=0.5, reg_alpha=0.05, subsample=0.8, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.5, reg_alpha=0.05, subsample=0.8, max_depth=5, n_estimators=500, total=  41.0s
[CV] colsample_bytree=0.7, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.7, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300, total=  22.0s
[CV] colsample_bytree=0.7, reg_alpha=0.01, subsample=0.6, max_depth=3, n_estimators=300 
[CV]  colsample_bytree=0.7, reg_alp

[CV]  colsample_bytree=0.7, reg_alpha=0.01, subsample=0.8, max_depth=5, n_estimators=500, total=  54.6s
[CV] colsample_bytree=0.7, reg_alpha=0.01, subsample=0.8, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.7, reg_alpha=0.01, subsample=0.8, max_depth=5, n_estimators=500, total=  57.4s
[CV] colsample_bytree=0.7, reg_alpha=0.01, subsample=0.8, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.7, reg_alpha=0.01, subsample=0.8, max_depth=5, n_estimators=500, total=  55.6s
[CV] colsample_bytree=0.7, reg_alpha=0.05, subsample=0.6, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.7, reg_alpha=0.05, subsample=0.6, max_depth=5, n_estimators=500, total=  41.2s
[CV] colsample_bytree=0.7, reg_alpha=0.05, subsample=0.6, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.7, reg_alpha=0.05, subsample=0.6, max_depth=5, n_estimators=500, total=  43.5s
[CV] colsample_bytree=0.7, reg_alpha=0.05, subsample=0.6, max_depth=5, n_estimators=500 
[CV]  colsample_bytree=0.7, reg_alp

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 51.8min finished


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=9999999999, n_estimators=500,
       nthread=-1, objective='binary:logistic', reg_alpha=0.05,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.6)


In [ ]:
# xgb_single = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
#        min_child_weight=1, missing=9999999999, n_estimators=300,
#        nthread=-1, objective='binary:logistic', reg_alpha=0.05,
#        reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
#        subsample=0.8)

# xgb_single.fit(train_X, train_y)
# valid_probs_xgb_single = xgb_single.predict_proba(valid_X)[:,1]
# valid_preds_xgb_single = xgb_single.predict(valid_X)
# print(accuracy_score(valid_y, valid_preds_xgb_single))
# print(roc_auc_score(valid_y, valid_probs_xgb_single))

In [45]:
valid_probs_rs = rs.predict_proba(valid_X.drop("CODE_GENDER_XNA", axis=1))[:,1]
valid_preds_rs= rs.predict(valid_X.drop("CODE_GENDER_XNA", axis=1))
print(accuracy_score(valid_y, valid_preds_rs))
print(roc_auc_score(valid_y, valid_probs_rs))

0.91489635638
0.67467142806


In [46]:
### Prepare submission file and save to disk
result_df = pd.DataFrame({'SK_ID_CURR':test.SK_ID_CURR.values, "TARGET":rs.predict_proba(test_feats)[:,1]})
result_df.to_csv("submimssion_3_xgboost_gridsearch_allfeatures_smote.csv", index=False)